# 01. Let's import necessary libraries

In [1]:
import os
import shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
import psutil

import sys
import pyemu
import flopy
import swatp_pst
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
assert "dependencies" in swatp_pst.__file__

In [2]:
swatp_pst.__version__

'0.0.0.dev0'

# 00. provide infos

In [3]:
# working directory
prjdir = "D:\\jj"
swatpwd = "D:\\Projects\\Tools\\swatp_pst_wf\\models\\TxtInOut_Imsil_rye_rot_r1"

In [12]:
# calibration period
cal_start = '1/1/2013'
cal_end = '12/31/2023'
# time step
time_step = 'day'
# locations (what our targets)
channels = [1]
obd_file = "singi_obs_q1_colnam.csv"
obd_colnam = "cha01"


# 01. inital setup

In [5]:
from swatp_pst import handler

In [6]:
handler.init_setup(prjdir, swatpwd)

failed to create 'main_opt' folder, folder already exists ...
path to main_opt folder: D:\jj\main_opt


In [7]:
wd = os.getcwd()

### Let's do initial run!

In [ ]:
pyemu.os_utils.run(r"swatplus.exe", cwd=".")

In [8]:
from swatp_pst.pst_utils import CreateIns, CreateTpl

# 02. Build template files

In [9]:
m1 = CreateTpl(wd)

In [10]:
m1.cal_to_tpl_file()

,cal_parm,chg_typ,chg_val,conds,soil_lyr1,soil_lyr2,yr1,yr2,day1,day2,obj_tot
cal_parm,,,,,,,,,,,
cn2,cn2,pctchg,~ cn2 ~,0,0,0,0,0,0,0,0
awc,awc,pctchg,~ awc ~,0,0,0,0,0,0,0,0
surlag,surlag,absval,~ surlag ~,0,0,0,0,0,0,0,0
alpha,alpha,absval,~ alpha ~,0,0,0,0,0,0,0,0
esco,esco,absval,~ esco ~,0,0,0,0,0,0,0,0


# 04. Build instruction files

In [11]:
# you need to run the model first if outputs do not exit.
# extract output first
# m1 = SWATpOut(wd)
m1.extract_mon_stf(channels, cal_start, cal_end)

stf_001.txt file has been created...
Finished ...


In [14]:
# m1.stf_obd_to_ins('stf_058.txt', 'rch058',cal_start, cal_end)
CreateIns.stf_obd_to_ins("stf_001.txt", obd_file, obd_colnam, obd_colnam, cal_start, cal_end)

AttributeError: 'str' object has no attribute 'read_cha_obd'

## 04.03 Depth to watertable (MODFLOW) 

In [ ]:
swatmf_pst_utils.extract_depth_to_water(grids, sim_start, cal_end)

In [ ]:
mf_obs_grid_ids = pd.read_csv(
                    'modflow.obs',
                    sep=r'\s+',
                    usecols=[3, 4],
                    skiprows=2,
                    header=None
                    )
sim_grids = mf_obs_grid_ids.iloc[:, 0].tolist()

In [ ]:
swatmf_pst_utils.extract_depth_to_water(sim_grids, sim_start, cal_end)

## 04.04 match it with modflow.obd file (MODFLOW)

In [ ]:
swatmf_pst_utils.mf_obd_to_ins('dtw_501.txt', 'g_5699', cal_start, cal_end)

# 05. Create PEST control file

In [ ]:
io_files = pyemu.helpers.parse_dir_for_io_files('.')
pst = pyemu.Pst.from_io_files(*io_files)

In [ ]:
par = pst.parameter_data
par

## 05.01 Assign parameter group name

In [ ]:
for i in range(len(par)):
    if (par.iloc[i, 0][:2]) == 'sy':
        par.iloc[i, 6] = 'sy'
    elif par.iloc[i, 0][:7] == 'rivbot_':
        par.iloc[i, 6] = 'rivbot'
    elif par.iloc[i, 0][:6] == 'rivcd_':
        par.iloc[i, 6] = 'rivcd'
    elif par.iloc[i, 0][:2] == 'hk':
        par.iloc[i, 6] = 'hk'
    else:
        par.iloc[i, 6] = 'swat'
print(par)

## 05.02 Adjust initial parameter values and their ranges

In [ ]:
count = 0
for i in range(len(par)):
    if (par.iloc[i, 6] == 'hk'):
        par.iloc[i, 3] = 1  
        par.iloc[i, 4] = 1.000000e-02
        par.iloc[i, 5] = 1.000000e+02
    elif (par.iloc[i, 6] == 'sy'):
        par.iloc[i, 3] = 1.000000e-02       
        par.iloc[i, 4] = 1.000000e-04
        par.iloc[i, 5] = 0.6  
    elif (par.iloc[i, 6] == 'rivbot'):
        par.iloc[i, 3] = 3.001     
        par.iloc[i, 4] = 0.001
        par.iloc[i, 5] = 6
        par.iloc[i, 8] = -3
    elif (par.iloc[i, 6] == 'rivcd'):
        par.iloc[i, 3] = 50.001       
        par.iloc[i, 4] = 0.001
        par.iloc[i, 5] = 100
        par.iloc[i, 8] = -50
    else:
        count += 1
count

In [ ]:
# CN2
par.loc['cn2', 'parval1'] = 1.001
par.loc['cn2', 'parlbnd'] = 0.8
par.loc['cn2', 'parubnd'] = 1.2
par.loc['cn2', 'offset'] = -1

# ESCO
par.loc['esco', 'parval1'] = 1.001
par.loc['esco', 'parlbnd'] = 0.5
par.loc['esco', 'parubnd'] = 1.5
par.loc['esco', 'offset'] = -1

# sol_awc()
par.loc['sol_awc()', 'parval1'] = 1.001
par.loc['sol_awc()', 'parlbnd'] = 0.5
par.loc['sol_awc()', 'parubnd'] = 1.5
par.loc['sol_awc()', 'offset'] = -1


## 05.03 Assign parameter group name

In [1]:
# set observation group
obd = pst.observation_data
obd

NameError: name 'pst' is not defined

In [ ]:
# Change obd group name
for i in range(len(obd)):
    obd.iloc[i, 3] = obd.iloc[i, 0][:-9]
obd

## 05.04 Provide actual observed values to control file

In [ ]:
os.getcwd()

In [ ]:
# Streamflow
stf_obd = pd.read_csv('stf_day.obd.csv',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
stf_obd = stf_obd[cal_start: cal_end]
stf_obd

In [ ]:
# watertable
dtw_obd = pd.read_csv('dtw_day.obd.csv',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
dtw_obd = dtw_obd[cal_start: cal_end]
dtw_obd

In [ ]:
# Get sub list based on obd order
obd_order = []
for i in obd.obgnme.tolist():
    if i not in obd_order:
        obd_order.append(i)
obd_order

In [ ]:
# get total list from each sub obd, delete na vals
tot_obd = []
for i in obd_order[:1]:
    tot_obd += dtw_obd[i].dropna().tolist()
    print(i)
for i in obd_order[1:]:
    tot_obd += stf_obd[i].dropna().tolist()
    print(i)
len(tot_obd)

In [ ]:
obd.loc[:, 'obsval'] = tot_obd
obd

# 06. Create the control file with settings

We can inspect all control data values using the `pst.control_data.formatted_values` attribute. Values are assigned defaults if not specified. Nice.:

In [ ]:
pst.control_data.formatted_values

In [ ]:
pst.control_data.noptmax = 0 # replace 0 with "zero" and see what happens
pst.model_command = 'python forward_run.py'

- ### add new PEST++ variables like so:

In [ ]:
# check the dictionary again
pst.pestpp_options

In [ ]:
pst.write('mb_zon.pst', version=2)

- ### You can also read and load the existing pest control file.

In [ ]:
pst_read = pyemu.Pst(os.path.join(swatmf_model,"mb_zon.pst"))

In [ ]:
pst_read.parameter_data